In [ ]:
# ============================================================
# E-step: seed weight 최적화(단순 simplex grid 0.05 간격)
#  - winsor=1.6, BINS=24, DROPOUT=0.06, TTA=14, PATIENCE=18
#  - Ridge(α inner-tune [2.6,2.8,3.0,3.2,3.4]) + Residual MLP + EMA
#  - fold trimmed mean(20%) → seed-weight grid search → 최종 예측
#  - Saves & downloads: /content/submission_YJ_Estep.csv
# ============================================================

import os, random, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import torch, torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

# ---------------- config ----------------
DATA_DIR = "/content/drive/MyDrive/인공지능 과제2"
SAVE_DIR = "/content"
SEEDS    = (42, 2025, 7, 11, 77)

BINS     = 24
INNER_ALPHAS = [2.6, 2.8, 3.0, 3.2, 3.4]

WINSOR  = 1.6
DROPOUT = 0.06
BETA    = 1.25
EMA_DEC = 0.996
TTA     = 14
BATCH   = 96
EPOCHS  = 220
PATIENCE= 18
TRIM_RATIO = 0.20
SOFT_CLIP = True

# ---------------- utils ----------------
def seed_all(s=42):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False

def rmse(y, p):
    y = np.asarray(y, dtype=np.float64).reshape(-1)
    p = np.asarray(p, dtype=np.float64).reshape(-1)
    return float(np.sqrt(np.mean((y - p) ** 2)))

def stratified_kfold_regression(y, n_splits=5, seed=42, n_bins=24):
    y = np.asarray(y).reshape(-1)
    q = int(min(n_bins, max(2, np.unique(y).size)))
    y_bins = pd.qcut(pd.Series(y), q=q, duplicates="drop").cat.codes.values
    rng = np.random.RandomState(seed)
    idx = rng.permutation(len(y))
    yb = y_bins[idx]
    buckets = [[] for _ in range(n_splits)]
    for b in np.unique(yb):
        b_idx = idx[yb == b]
        for k, j in enumerate(range(len(b_idx))):
            buckets[k % n_splits].append(b_idx[j])
    folds = []
    for k in range(n_splits):
        va_idx = np.array(sorted(buckets[k]))
        tr_mask = np.ones(len(y), dtype=bool); tr_mask[va_idx] = False
        tr_idx = np.where(tr_mask)[0]
        folds.append((tr_idx, va_idx))
    return folds

def interactions_min(df: pd.DataFrame):
    df = df.copy()
    if "bw" in df.columns and "b.head" in df.columns:
        df["bw__x__b.head"] = df["bw"] * df["b.head"]
        df["head_bw_ratio"] = df["b.head"] / (df["bw"] + 1e-6)
    if "preterm" in df.columns and "bw" in df.columns:
        df["preterm__x__bw"] = df["preterm"] * df["bw"]
    if "nnhealth" in df.columns and "b.head" in df.columns:
        df["nnhealth__x__b.head"] = df["nnhealth"] * df["b.head"]
    return df

def trimmed_mean(arr, trim_ratio=0.2, axis=0):
    arr = np.sort(arr, axis=axis)
    n = arr.shape[axis]
    k = int(n * trim_ratio)
    if n - 2*k <= 0:
        return arr.mean(axis=axis)
    if axis == 0:
        return arr[k:n-k].mean(axis=0)
    else:
        return arr[:, k:n-k].mean(axis=1)

def soft_clip_to_iqr(pred, y_train, verbose=True):
    q1, q3 = np.percentile(y_train, [25, 75])
    iqr = q3 - q1
    lo = q1 - 1.5 * iqr
    hi = q3 + 1.5 * iqr
    if verbose:
        print(f"[Clip-range] lo={lo:.3f}, hi={hi:.3f}")
    return np.clip(pred, lo, hi)

# ---------------- model ----------------
class SmallMLP(nn.Module):
    def __init__(self, d, sizes=(64,32), dropout=0.10):
        super().__init__()
        h1, h2 = sizes
        self.seq = nn.Sequential(
            nn.Linear(d, h1), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, 1)
        )
    def forward(self, x): return self.seq(x)

class EMA:
    def __init__(self, model, decay=0.996):
        self.decay = decay
        self.shadow = {k: v.detach().clone() for k,v in model.state_dict().items()}
    def update(self, model):
        with torch.no_grad():
            for k, v in model.state_dict().items():
                self.shadow[k].mul_(self.decay).add_(v.detach(), alpha=1-self.decay)
    def copy_to(self, model): model.load_state_dict(self.shadow)

@torch.no_grad()
def predict_mc(model, X, tta=TTA):
    model.train()  # dropout on
    outs = []
    for _ in range(tta):
        outs.append(model(X).cpu().numpy().reshape(-1))
    return np.mean(np.stack(outs, axis=0), axis=0)

# ---------------- one fold train ----------------
def train_residual_fold_YJ(
    X_tr, y_tr, X_va, y_va, X_te,
    inner_alphas=INNER_ALPHAS, beta=BETA, lr=6e-4, weight_decay=1e-4,
    batch_size=BATCH, epochs=EPOCHS, patience=PATIENCE, mlp_sizes=(64,32), dropout=DROPOUT,
    winsor_pct=WINSOR, ema_decay=EMA_DEC, tta=TTA
):
    # 0) winsorize
    if winsor_pct and winsor_pct > 0:
        lo, hi = np.percentile(y_tr, [winsor_pct, 100 - winsor_pct])
        y_tr_fit = y_tr.clip(lo, hi)
    else:
        y_tr_fit = y_tr

    # α mini-tune (3-fold)
    best_a, best_rm = None, 1e9
    kf_in = KFold(n_splits=3, shuffle=True, random_state=42)
    for a in inner_alphas:
        scs=[]
        for it_tr, it_va in kf_in.split(X_tr):
            X_it_tr, X_it_va = X_tr.iloc[it_tr], X_tr.iloc[it_va]
            y_it_tr, y_it_va = y_tr_fit.iloc[it_tr], y_tr.iloc[it_va]
            yt_pt_inner = PowerTransformer(method="yeo-johnson", standardize=True)
            y_it_tr_t = yt_pt_inner.fit_transform(y_it_tr.values.reshape(-1,1)).reshape(-1)
            pipe = Pipeline([
                ("pt", PowerTransformer(method="yeo-johnson", standardize=False)),
                ("sc", StandardScaler()),
                ("rg", Ridge(alpha=a, random_state=42)),
            ])
            pipe.fit(X_it_tr, y_it_tr_t)
            pred_t = pipe.predict(X_it_va)
            pred   = yt_pt_inner.inverse_transform(pred_t.reshape(-1,1)).reshape(-1)
            scs.append(rmse(y_it_va, pred))
        m = float(np.mean(scs))
        if m < best_rm: best_rm, best_a = m, a

    # 본학습: Ridge(YJ target) + residual MLP
    ridge_pipe = Pipeline([
        ("pt", PowerTransformer(method="yeo-johnson", standardize=False)),
        ("sc", StandardScaler()),
        ("rg", Ridge(alpha=best_a, random_state=42)),
    ])
    yt_pt = PowerTransformer(method="yeo-johnson", standardize=True)
    y_tr_t = yt_pt.fit_transform(y_tr_fit.values.reshape(-1,1)).reshape(-1)

    ridge_pipe.fit(X_tr, y_tr_t)
    yhat_tr_t = ridge_pipe.predict(X_tr)
    yhat_va_t = ridge_pipe.predict(X_va)
    yhat_te_t = ridge_pipe.predict(X_te)

    resid_tr = y_tr_t - yhat_tr_t
    Xtr_tf = ridge_pipe.named_steps["sc"].transform(ridge_pipe.named_steps["pt"].transform(X_tr))
    Xva_tf = ridge_pipe.named_steps["sc"].transform(ridge_pipe.named_steps["pt"].transform(X_va))
    Xte_tf = ridge_pipe.named_steps["sc"].transform(ridge_pipe.named_steps["pt"].transform(X_te))

    mu, sd = resid_tr.mean(), resid_tr.std()
    rtr_z = (resid_tr - mu) / (sd + 1e-8)

    Xtr_t = torch.tensor(Xtr_tf, dtype=torch.float32)
    rtr_t = torch.tensor(np.asarray(rtr_z).reshape(-1,1), dtype=torch.float32)
    Xva_t = torch.tensor(Xva_tf, dtype=torch.float32)
    Xte_t = torch.tensor(Xte_tf, dtype=torch.float32)

    model = SmallMLP(d=Xtr_t.shape[1], sizes=mlp_sizes, dropout=dropout)
    try:
        crit = nn.SmoothL1Loss(beta=beta)
    except TypeError:
        crit = nn.L1Loss()
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    sch = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode="min", factor=0.5, patience=10)
    ema = EMA(model, decay=ema_decay)

    best, wait, best_state = 1e9, 0, None
    for ep in range(1, epochs+1):
        model.train()
        for xb, yb in DataLoader(TensorDataset(Xtr_t, rtr_t), batch_size=batch_size, shuffle=True):
            opt.zero_grad(); loss = crit(model(xb), yb); loss.backward(); opt.step()
            ema.update(model)
        # valid with EMA
        ema.copy_to(model); model.eval()
        with torch.no_grad():
            rz_va = model(Xva_t).cpu().numpy().reshape(-1)
        r_va_t = rz_va*(sd+1e-8) + mu
        pred_va_t = yhat_va_t + r_va_t
        pred_va = yt_pt.inverse_transform(pred_va_t.reshape(-1,1)).reshape(-1)
        sc = rmse(y_va, pred_va)
        sch.step(sc)
        if sc < best - 1e-6:
            best = sc; best_state = {k: v.detach().clone() for k,v in model.state_dict().items()}; wait=0
        else:
            wait += 1
        if wait >= patience: break

    # best → OOF/TEST (MC Dropout)
    model.load_state_dict(best_state); ema.copy_to(model)
    with torch.no_grad():
        rz_va = predict_mc(model, Xva_t, tta=tta)
        r_va_t = rz_va*(sd+1e-8) + mu
        oof_t  = yhat_va_t + r_va_t
        oof    = yt_pt.inverse_transform(oof_t.reshape(-1,1)).reshape(-1)

        rz_te = predict_mc(model, Xte_t, tta=tta)
        r_te_t = rz_te*(sd+1e-8) + mu
        te_t   = yhat_te_t + r_te_t
        te     = yt_pt.inverse_transform(te_t.reshape(-1,1)).reshape(-1)

    return oof, te, best, best_a

def interactions_min_all(train_df, test_df, feat_cols):
    X  = interactions_min(train_df[feat_cols].copy())
    Xt = interactions_min(test_df[feat_cols].copy())
    return X, Xt

# ---------------- seed weight grid search ----------------
def simplex_weight_grid(n=5, step=0.05):
    # 생성: w_i >=0, sum=1
    vals = np.arange(0, 1+1e-9, step)
    weights = []
    def rec(prefix, remain, k):
        if k == n-1:
            w = prefix + [remain]
            if remain >= -1e-9:
                weights.append(np.array(w, dtype=np.float64))
            return
        for v in vals:
            if v <= remain + 1e-9:
                rec(prefix + [v], remain - v, k+1)
    rec([], 1.0, 0)
    return weights

# ---------------- main ----------------
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)

# data
train = pd.read_csv(f"{DATA_DIR}/train.csv")
test  = pd.read_csv(f"{DATA_DIR}/test.csv")
sub   = pd.read_csv(f"{DATA_DIR}/sample_submission.csv")
if "dadage" in train.columns:
    train["dadage"] = train["dadage"].fillna(train["dadage"].mean())
    test["dadage"]  = test["dadage"].fillna(train["dadage"].mean())

FEATS = [c for c in train.columns if c not in ["id","y"]]
X_raw = train[FEATS].copy(); y = train["y"].copy(); X_te_raw = test[FEATS].copy()
X, X_te = interactions_min_all(train, test, FEATS)

# seeds×fold
seed_preds = []; seed_oofs = []; seed_scores = []; seed_alpha_means = []
for s in SEEDS:
    seed_all(s)
    folds = stratified_kfold_regression(y, n_splits=5, seed=s, n_bins=BINS)
    oof = np.zeros(len(X)); te_fold=[]; alphas=[]
    for tr_idx, va_idx in folds:
        X_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
        X_va, y_va = X.iloc[va_idx], y.iloc[va_idx]
        o, t, sc, a = train_residual_fold_YJ(
            X_tr, y_tr, X_va, y_va, X_te,
            inner_alphas=INNER_ALPHAS, beta=BETA, dropout=DROPOUT,
            winsor_pct=WINSOR, ema_decay=EMA_DEC, tta=TTA,
            batch_size=BATCH, epochs=EPOCHS, patience=PATIENCE
        )
        oof[va_idx]=o; te_fold.append(t); alphas.append(a)
    sc_seed = rmse(y, oof)
    te_fold = np.vstack(te_fold)
    pred_seed = trimmed_mean(te_fold, trim_ratio=TRIM_RATIO, axis=0)
    seed_preds.append(pred_seed)
    seed_oofs.append(oof)
    seed_scores.append(sc_seed)
    seed_alpha_means.append(np.mean(alphas))
    print(f"seed {s} OOF={sc_seed:.5f} (α~{np.mean(alphas):.2f})")
print("OOF per seed:", [f"{v:.5f}" for v in seed_scores])

oof_mat   = np.vstack(seed_oofs)   # (S, N)
pred_mat  = np.vstack(seed_preds)  # (S, N)

# ---- seed weight grid search (simplex step=0.05) ----
cands = simplex_weight_grid(n=len(SEEDS), step=0.05)
best = {"w": None, "rmse": 1e9}
y_np = y.values
for w in cands:
    oof_w  = np.sum(oof_mat.T  * w, axis=1)
    sc = rmse(y_np, oof_w)
    if sc < best["rmse"]:
        best = {"w": w, "rmse": sc}
print(f"[WeightSearch] best OOF RMSE={best['rmse']:.5f} | w={np.round(best['w'],3)}")

# apply best weights
w_best  = best["w"]
oof_ens = np.sum(oof_mat.T  * w_best, axis=1)
pred_ens= np.sum(pred_mat.T * w_best, axis=1)

print(f"[Ensemble] OOF RMSE: {rmse(y_np, oof_ens):.5f}")

# optional soft-clip
def soft_clip_to_iqr(pred, y_train, verbose=True):
    q1, q3 = np.percentile(y_train, [25, 75])
    iqr = q3 - q1
    lo = q1 - 1.5 * iqr
    hi = q3 + 1.5 * iqr
    if verbose:
        print(f"[Clip-range] lo={lo:.3f}, hi={hi:.3f}")
    return np.clip(pred, lo, hi)

pred_final = soft_clip_to_iqr(pred_ens, y_np, verbose=True) if SOFT_CLIP else pred_ens

# save & download
sub_out = sub.copy(); sub_out["y"] = pred_final
out_path = os.path.join(SAVE_DIR, "submission_YJ_Estep.csv")
sub_out.to_csv(out_path, index=False)
print(f"✅ Saved -> {out_path}")

files.download(out_path)